In [23]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
from matplotlib.pyplot import hist
from scipy import stats
# import more functions or modules if you need them !!

In [24]:
# set random seed for numpy
RANDOM_SEED=42
np.random.seed(RANDOM_SEED)

In [25]:
oil_df = pd.read_pickle('../Data/data/comprehensive_new_full.pkl')

In [26]:
oil_df

,year,numcode,oilreserves_full,oilreserves,oilreserves_public,newdiscovery_aspo,aspo,wildcat,endowment,pop_maddison,...,valoilres_public_binarize,oilpop_diff,oilpop_binarize,valoilres_impute_diff,valoilres_impute_binarize,oilpop_impute_diff,oilpop_impute_binarize,crude1990P_diff,milexp_pergdpSIPRI_diff,newdiscovery_aspo_binarize
0,1929.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1931.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.401178,NaN,NaN
3,1932.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.492896,NaN,NaN
4,1933.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.188141,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17915,2004.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,10962.026367,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.292683,NaN,NaN
17916,2005.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11115.380859,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.378049,NaN,NaN
17917,2006.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11288.252930,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.157455,NaN,NaN
17918,2007.0,894,0.00001,NaN,0.00001,NaN,NaN,NaN,0.00001,11477.447266,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.080496,NaN,NaN


In [5]:
oil_df.columns

Index(['year', 'numcode', 'oilreserves_full', 'oilreserves',
       'oilreserves_public', 'newdiscovery_aspo', 'aspo', 'wildcat',
       'endowment', 'pop_maddison',
       ...
       'valoilres_public_binarize', 'oilpop_diff', 'oilpop_binarize',
       'valoilres_impute_diff', 'valoilres_impute_binarize',
       'oilpop_impute_diff', 'oilpop_impute_binarize', 'crude1990P_diff',
       'milexp_pergdpSIPRI_diff', 'newdiscovery_aspo_binarize'],
      dtype='object', length=119)

In [6]:
# ATT and ATE AIPTW
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    Return: aiptw of ATE and its standard error
    """
    
    # number of observations
    n = Y.shape[0]
    
    # estimate marginal probability of treatment
    if prob_t is None:
        prob_t = A.mean() 
    
    # att aiptw
    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    # influence curve and standard error of aiptw
    phi = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATE
    Return: aiptw of ATE and its standard error
    """
    # number of observations
    n = Y.shape[0]
    
    # ate aiptw
    tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
    # influence curve and standard error of aiptw
    phi = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat   
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

In [7]:
# Conditional outcome models (Q models)
def make_linear_Q_model():
    ''' A function that returns a linear q model for later use in k-folding'''
    return LinearRegression()

def make_Q_model(output_type:str):
    ''' A function that returns a general ML q model for later use in k-folding'''
    if output_type == 'binary':
        return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
# One example: RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)

In [8]:
# Propensity score models (g models)
def make_g_model():
    ''' A function that returns a g model for computing propensity scores'''
    return RandomForestClassifier(n_estimators=100, max_depth=5)
# One example: RandomForestClassifier(n_estimators=100, max_depth=5)

In [9]:
# Functions for K-fold cross-fitting
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    '''
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    '''

    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    print(len(A.val))
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]
    
    # sanity check that overlap holds
    assert np.isnan(predictions).sum() == 0
    return predictions

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"
    '''

    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model(output_type)
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [11]:
model_vars = ['ecgrowth',
              'pop_maddison_diff',
              'popdens_diff',
              'democracy_diff',
              'logmountain',
              'ethnic_fractionalization',
              'religion_fractionalization',
              'language_fractionalization',
              'leg_british',
              'decade']

In [12]:
outcome = 'onset2COWCS'
treatment = 'valoilres_binarize'
confounders = [x for x in model_vars if x not in (outcome + treatment)]

In [56]:
confounders

['ecgrowth',
 'pop_maddison_diff',
 'popdens_diff',
 'democracy_diff',
 'logmountain',
 'ethnic_fractionalization',
 'religion_fractionalization',
 'language_fractionalization',
 'leg_british',
 'decade']

In [36]:
from sklearn.preprocessing import LabelEncoder

In [87]:
def fit_and_run_model(df, outcome:str, treatment:str, confounders:list, make_g_model,
                      make_Q_model, n_splits=5, output_type='binary', ate=True):
    '''
    Function that creates a g, q, and aiptw model based on the 
    given inputs
    
    Inputs: df (pandas df) - the dataframe the variables are contained in
            outcome (str) - the outcome variable
            treatment (str) - the treatment variable
            confounders (lst) - a list of the confounding variables
            make_g_model - the make_g_model function
            make_Q_model - the make_Q_model function
            n_splits (int) - number of splits for the model
            output_type (str) - the desired output type, either binary or continous
            ate (bool) - whether to use ate or alternative att
    
    Returns: tau_hat - the tau hat estimator for the average treatment effect
             std of tau_hat - the standard deviation for the tau_hat estimator
    '''
    df = df.replace({outcome: .00001}, 0)
    df = df[[outcome] + confounders + [treatment]]
    df = df.dropna().reset_index()
    print('Running models for treatment {} and outcome {} on {} samples'.format(treatment, outcome, len(df)))
    n = len(df)
    outcome = df[outcome]
    confounders = df[confounders]
    treatment = df[treatment]
    treatment = treatment.replace({0.0: 0, 1.0: 1})
    outcome = outcome.replace({0.0: 0, 1.0: 1})
    g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=n_splits)
    drop = False
    if min(g) < .01:
        print('\nWARNING:\n Some propensity scores are very small,\n which could '
              'lead to an inflated AIPTW.\n Minimum score = ', min(g))
        #drop = True
    if max(g) > .99:
        print('\nWARNING:\n Some propensity scores are very large,\n which could '
              'lead to an inflated AIPTW.\n Maximum score = ', max(g))
       #drop = True
    print('G Model has been fit')

    Q0_ml, Q1_ml = outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, \
                                                  n_splits=n_splits, output_type=output_type)
    
    print('Q model has been fit')
    data_and_nuisance_estimates_ml = pd.DataFrame({'g': g, 'Q0': Q0_ml, 'Q1': Q1_ml, 'A': treatment, 'Y': outcome})
    
    if drop:
        data_and_nuisance_estimates_ml = data_and_nuisance_estimates_ml[data_and_nuisance_estimates_ml['g'] > .01]
        data_and_nuisance_estimates_ml = data_and_nuisance_estimates_ml[data_and_nuisance_estimates_ml['g'] < .99]
        print('Dropped {} observations due to overlap condition'.format(len(df) - len(data_and_nuisance_estimates_ml)))
    # ate aiptw
    if ate:
        tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates_ml)
    else: 
        tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates_ml)
    test_stat = tau_hat / std_hat
    p_value = stats.t.sf(abs(test_stat), df=(n-1))
    print('AIPTW model has been fit. Returning \u03C4 hat and its standard deviation')
    print('\u03C4 hat = {}, std = {}, test statistic = {}, p-value = {}\n'.format(round(tau_hat, 5), round(std_hat, 5), round(test_stat, 5), round(p_value, 5)))
    return tau_hat, std_hat, test_stat, p_value

In [88]:
treatment_lst = ['valoilres_binarize', # value of oil reserves
               'valoilres_public_binarize', # value of oil reserves from public data
               'oilpop_binarize', # oil reserves per capita in million barrels per 1000 persons
               'valoilres_impute_binarize', # value of oilpop_impute (multiply by crude oil price)
               'oilpop_impute_binarize']

outcome_lst = ['onset2COWCS',
               'onsetUCS',
               'coup',
               'periregular',
               'milexp_pergdpSIPRI_diff']

In [89]:
oil_df = oil_df.T.drop_duplicates().T

In [90]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': [], 'test_stat': [], 'p_value': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std, test_stat, p_value = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output, ate=False)
        df_dict['model'].append('att')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)
        df_dict['test_stat'].append(test_stat)
        df_dict['p_value'].append(p_value)

att_results_df = pd.DataFrame(data=df_dict)

Running models for treatment valoilres_binarize and outcome onset2COWCS on 5174 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00265, std = 0.00496, test statistic = 0.53366, p-value = 0.2968

Running models for treatment valoilres_binarize and outcome onsetUCS on 4754 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00857, std = 0.00855, test statistic = 1.00159, p-value = 0.1583

Running models for treatment valoilres_binarize and outcome coup on 5364 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.01248, std = 0.01321, test statistic = -0.94414, p-value = 0.17257

Running models for treatment valoilres_binarize and outcome periregular on 3312 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its st

In [91]:
att_results_df

,model,treatment,outcome,tau_hat,tau_std,test_stat,p_value
0,att,valoilres_binarize,onset2COWCS,0.002646,0.004958,0.533655,0.296801
1,att,valoilres_binarize,onsetUCS,0.008568,0.008555,1.001591,0.158296
2,att,valoilres_binarize,coup,-0.012476,0.013214,-0.944137,0.172571
3,att,valoilres_binarize,periregular,-0.013784,0.005624,-2.450894,0.007151
4,att,valoilres_binarize,milexp_pergdpSIPRI_diff,0.656899,0.431134,1.523653,0.063919
5,att,valoilres_public_binarize,onset2COWCS,0.001143,0.004156,0.274943,0.391686
6,att,valoilres_public_binarize,onsetUCS,0.019144,0.007177,2.667453,0.003834
7,att,valoilres_public_binarize,coup,0.019410,0.011467,1.692622,0.045292
8,att,valoilres_public_binarize,periregular,-0.005928,0.007504,-0.789895,0.214831
9,att,valoilres_public_binarize,milexp_pergdpSIPRI_diff,0.388155,0.275646,1.408163,0.079643


In [92]:
att_results_df[att_results_df['p_value'] < .05]

,model,treatment,outcome,tau_hat,tau_std,test_stat,p_value
3,att,valoilres_binarize,periregular,-0.013784,0.005624,-2.450894,0.007151
6,att,valoilres_public_binarize,onsetUCS,0.019144,0.007177,2.667453,0.003834
7,att,valoilres_public_binarize,coup,0.019410,0.011467,1.692622,0.045292
16,att,valoilres_impute_binarize,onsetUCS,0.017723,0.008022,2.209423,0.013596
18,att,valoilres_impute_binarize,periregular,-0.013735,0.005597,-2.453875,0.007092


In [97]:
treatment_lst = ['newdiscovery_aspo_binarize']
outcome_lst = ['d2incidenceU',
               'd3_6incidenceU',
               'onset2COWCS',
               'onsetUCS',
               'coup',
               'periregular',
               'milexp_pergdpSIPRI_diff']
confoundes = ['crude1990P_diff',
               'wildcat_diff',
               'out_regdisaster_diff',
               'ecgrowth',
               'pop_maddison_diff',
               'popdens_diff',
               'democracy_diff',
               'logmountain',
               'ethnic_fractionalization',
               'religion_fractionalization',
               'language_fractionalization',
               'leg_british',
               'decade',
               'opec']

In [98]:
df_dict = {'model': [], 'treatment': [], 'outcome': [], 'tau_hat': [], 'tau_std': [], 'test_stat': [], 'p_value': []}
for treat in treatment_lst:
    for out in outcome_lst:
        if len(oil_df[out].value_counts()) == 2:
            output = 'binary'
        else:
            output = 'continuous'
        tau_hat, tau_std, test_stat, p_value = fit_and_run_model(oil_df, out, treat, confounders, make_g_model, make_Q_model, output_type=output, ate=False)
        df_dict['model'].append('att')
        df_dict['treatment'].append(treat)
        df_dict['outcome'].append(out)
        df_dict['tau_hat'].append(tau_hat)
        df_dict['tau_std'].append(tau_std)
        df_dict['test_stat'].append(test_stat)
        df_dict['p_value'].append(p_value)

att_discovery_results_df = pd.DataFrame(data=df_dict)

Running models for treatment newdiscovery_aspo_binarize and outcome d2incidenceU on 2561 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00752, std = 0.0138, test statistic = 0.54534, p-value = 0.29278

Running models for treatment newdiscovery_aspo_binarize and outcome d3_6incidenceU on 2509 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = -0.01196, std = 0.01556, test statistic = -0.76843, p-value = 0.22115

Running models for treatment newdiscovery_aspo_binarize and outcome onset2COWCS on 4977 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard deviation
τ hat = 0.00486, std = 0.00399, test statistic = 1.21862, p-value = 0.11152

Running models for treatment newdiscovery_aspo_binarize and outcome onsetUCS on 4307 samples
G Model has been fit
Q model has been fit
AIPTW mo

In [99]:
att_discovery_results_df

,model,treatment,outcome,tau_hat,tau_std,test_stat,p_value
0,att,newdiscovery_aspo_binarize,d2incidenceU,0.007524,0.013798,0.545340,0.292783
1,att,newdiscovery_aspo_binarize,d3_6incidenceU,-0.011955,0.015558,-0.768430,0.221152
2,att,newdiscovery_aspo_binarize,onset2COWCS,0.004862,0.003990,1.218622,0.111523
3,att,newdiscovery_aspo_binarize,onsetUCS,0.009588,0.006034,1.589157,0.056049
4,att,newdiscovery_aspo_binarize,coup,-0.026997,0.009857,-2.738904,0.003093
5,att,newdiscovery_aspo_binarize,periregular,-0.013879,0.004177,-3.322793,0.000450
6,att,newdiscovery_aspo_binarize,milexp_pergdpSIPRI_diff,0.146920,0.162192,0.905838,0.182595
